### Workshop Tutorial: Note that all slides have been optimized for a [RISE slideshow](https://rise.readthedocs.io)

In [1]:
# Ignore warnings that distract from the tutorial
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

<div align='center' style='font-size:200%'>Validating Computational Models with</div>
<div align='center'><img src="https://raw.githubusercontent.com/scidash/assets/master/logos/SciUnit/sci-unit-wide.png" width="50%"></div>
<hr>
<br>
<div align='center' style='font-size:150%; line-height:80px;'>
Richard C. (Rick) Gerkin, PhD
</div>    
<div align='center' style='font-size:100%; line-height:80px;'>Associate Research Professor, School of Life Sciences<br>
Co-Director, <a href="http://iconlab.asu.edu">Laboratory for Informatics and Computation in Open Neuroscience</a><br>
Arizona State University, Tempe, AZ USA
</div>
<hr>

### <i>SciUnit</i> is a Python package.  Let's make sure it's installed.

In [2]:
# An object-oriented Python framework for model validation
import sciunit

### <i>SciUnit</i> is a framework for validating scientific models by creating experimental-data-driven <i>unit tests</i>.

#### Conventionally, a unit test is a <i>"a strict, written contract that the piece of code must satisfy</i>”

#### <i>SciUnit</i> extends this to scientific models, making model validation both formal and transparent.  

#### The validity of a model is then represented by the collection of unit tests that it passes.

In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
    
    td.red {
        background-color: #FF0000;
        border: 1px solid black;
    }
    td.green {
        background-color: #00FF00;
        border: 1px solid black;
    }
    td.grey {
        background-color: #AAAAAA;
        border: 1px solid black;
    }
    th {
        text-align: center;
        border: 1px solid black;
    }
    table td, table th {
    border: 10px solid black;
    font-size: 250%;
    }
</style>
"""))

<div style='font-size:200%; text-align: center;'>Abstract example: A brief history of cosmology</div><br>

<table>
<tr style='background-color: #FFFFFF'>
    <td></td><th style='text-align: center'>Experimentalists</th>
</tr>
<tr>
    <th>Modelers</th><td><table style='border: 1px solid black'></td>
<tr>
    <th></th><th>Babylonians</th><th>Brahe</th><th>Galileo</th><th>Le Verrier</th>
</tr>
<tr>
    <th>Ptolemy</th><td class='green'></td><td class='red'></td><td class='red'></td><td class='red'></td>
</tr>
<tr>
    <th>Copernicus</th><td class='green'></td><td class='red'></td><td class='red'></td><td class='red'></td>
</tr>
<tr>
    <th>Kepler</th><td class='green'></td><td style='background-color:#FF0000'></td><td class='grey'></td><td class='red'></td>
</tr>
<tr>
    <th>Newton</th><td class='green'></td><td class='green'></td><td class='green'></td><td class='red'></td>
</tr>
<tr>
    <th>Einstein</th><td class='green'></td><td class='green'></td><td class='green'></td><td class='green'></td>
</tr>
</table>
</td>
</tr>
</table>

<table style='border: 1px solid black'>
    <tr>
        <td class='green'>Pass</td><td class='red'>Fail</td><td class='grey'>Unclear</td>
    </tr>
</table>

## Goals:   

### - Generate such unit tests for any experimental data

### - Execute these tests against all models capable of taking them

### - Programatically display the results as a 'dashboard' of model validity
  - Optionally record and display non-boolean test results, test artifacts, etc.

### High-level workflow for validation:

```python
# Hypothetical examples of data-driven tests
from cosmosuite.tests import brahe_test, galileo_test, leverrier_test

# Hypothetical examples of parameterized models
from cosmosuite.models import ptolemy_model, copernicus_model 

# Execute one test against one model and return a test score
brahe_test.judge(copernicus_model) 
```

### Q: How does a test 'know' how to test a model?

### A: Through guarantees that models provide to tests, called <i>'Capabilities'</i>.

[Code for **sciunit.capabilities** on GitHub](https://github.com/scidash/sciunit/tree/master/sciunit/capabilities.py)

### Next we show an example of a Capability relevant to the cosmology example

In [4]:
from math import pi, sqrt, sin, cos, tan, atan
from datetime import datetime, timedelta
from sympy import Symbol, solvers, sin as sin_
import numpy as np

In [5]:
class ProducesOrbitalPosition(sciunit.Capability):
    """
    A model `capability`, i.e. a collection of methods
    that a test is allowed to invoke on a model.
    These methods are unimplemented by design,
    and the model must implement them.
    """
    
    def get_position(self, t: datetime) -> tuple:
        """Produce an orbital position from a time point
        
        Args:
            t (datetime): The time point to examine, 
                          relative to perihelion

        Returns:
            tuple: A pair of (r, theta) coordinates
                   in the oribtal plane
        """
        raise NotImplementedError("")
        
    @property
    def perihelion(self) -> datetime:
        """Return the time of last perihelion"""
        raise NotImplementedError("")
    
    @property
    def period(self) -> float:
        """Return the period of the orbit"""
        raise NotImplementedError("")
    
    @property
    def eccentricity(self) -> float:
        """Return the eccentricity of the orbit"""
        raise NotImplementedError("")
    
    def get_x_y(self, t: datetime) -> tuple:
        r, theta = self.get_position(t)
        x = r*cos(theta)
        y = r*sin(theta)
        return x, y

### <i>SciUnit</i> (and domain specific libraries that build upon it) also define their own capabilities

In [6]:
# An extremely generic model capability
from sciunit.capabilities import ProducesNumber

# A specific model capability used in neurophysiology
#from neuronunit.capabilities import HasMembranePotential

### Now we can define a <i>model class</i> that implements this `ProducesOrbitalPosition` capability by inheritance.  

### All models are subclasses of `sciunit.Model` and typically one or more subclasses of `sciunit.Capability`.

In [ ]:
class BaseKeplerModel(sciunit.Model, 
                      ProducesOrbitalPosition):
    """A sciunit model class corresponding to a Kepler-type model
    of an object in the solar system. This model has the 
    `ProducesOrbitalPosition` capability by inheritance"""
    
    def get_position(self, t):
        r, theta = self.heliocentric_distance(t), self.true_anomaly(t)
        return r, theta
    
    @property
    def perihelion(self):
        """Implementation of time of last perihelion"""
        return self.params['perihelion']
    
    @property
    def period(self):
        """Implementation of period of the orbit"""
        return self.params['period']
    
    @property
    def eccentricity(self):
        """Implementation of orbital eccentricity (assuming elliptic orbit)"""
        a, b = self.params['semimajor_axis'], self.params['semiminor_axis']
        return sqrt(1 - (b/a)**2)

In [7]:
class KeplerModel(BaseKeplerModel):
    """This 'full' model contains all of the methods required
    to complete the implementation of the `ProducesOrbitalPosition` capability"""
    
    def mean_anomaly(self, t):
        """How long into its period the object is at time `t`"""
        time_since_perihelion = t - self.perihelion
        return 2*pi*(time_since_perihelion % self.period)/self.period
    
    def eccentric_anomaly(self, t):
        """How far the object has gone into its period at time `t`"""
        E = Symbol('E')
        M, e = self.mean_anomaly(t), self.eccentricity
        expr = E - e*sin_(E) - M
        return solvers.nsolve(expr, 0)
    
    def true_anomaly(self, t):
        """Theta in a polar coordinate system at time `t`"""
        e, E = self.eccentricity, self.eccentric_anomaly(t)
        theta = 2*atan(sqrt(tan(E/2)**2 * (1+e)/(1-e)))
        return theta
    
    def heliocentric_distance(self, t):
        """R in a polar coordinate system at time `t`"""
        a, e = self.params['semimajor_axis'], self.eccentricity
        E = self.eccentric_anomaly(t)
        return a*(1-e*cos(E))

### Now we can instantiate a <i>specific model</i> from this class, e.g. one representing orbital path of Earth (according to Kepler)

In [8]:
import quantities as pq
earth_model = KeplerModel(name = "Kepler's Earth Model",
                          semimajor_axis=149598023 * pq.km, 
                          semiminor_axis=149577161 * pq.km, 
                          period=timedelta(365, 22118), # Period of Earth's orbit 
                          perihelion=datetime(2019, 1, 3, 0, 19), # Time and date of Earth's last perihelion
                         )

### We can use this model to make specific predictions, for example the current distance between Earth and the sun.

In [9]:
t = datetime.now()
r = earth_model.heliocentric_distance(t)
print("Heliocentric distance of Earth right now is predicted to be %s" % r.round(1))

Heliocentric distance of Earth right now is predicted to be 151709603.8 km


### Now let's build a test class that we might use with this (and hopefully other) models

### First, what kind of scores do we want our test to return?

In [10]:
# Several score types available in SciUnit
from sciunit.scores import BooleanScore, ZScore, RatioScore, PercentScore # etc., etc.

[Code for **sciunit.scores** on GitHub](https://github.com/scidash/sciunit/tree/master/sciunit/scores)

### Here's a first shot a test class for assessing the agreement between predicted and observed positions of orbiting objects.  All test classes are subclasses of `sciunit.Test`.

In [51]:
class PositionTest(sciunit.Test):
    """A test of a planetary position at some specified time"""
    
    # This test can only operate on models that implement
    # the `ProducesOrbitalPosition` capability.
    required_capabilities = (ProducesOrbitalPosition,)
    
    # This test's 'judge' method will return a BooleanScore.
    score_type = BooleanScore 
    
    def generate_prediction(self, model):
        """Generate a prediction from a model"""
        # Get the time point from the test's observation
        t = self.observation['t']
        # Get the predicted x, y coordinates from the model
        x, y = model.get_x_y(t)
        # Roll this into a model prediction dictionary
        return {'t': t, 'x': x, 'y': y}
        
    def compute_score(self, observation, prediction):
        # Compare observation and prediction to get an error measure
        delta_x = observation['x'] - prediction['x']
        delta_y = observation['y'] - prediction['y']
        error = np.sqrt(delta_x**2 + delta_y**2)
        
        # Turn this into a True/False score
        passing = bool(error < 1e5*pq.kilometer)
        score = self.score_type(passing)
        score.set_raw(error)
        score.description = ("Passing score if the prediction is "
                             "within < 100,000 km of the observation")
        return score

### We might want to include extra checks and constraints on observed data, test parameters, or other contingent testing logic.

In [52]:
class StricterPositionTest(PositionTest):
    # Optional observation units to validate against
    units = pq.meter
    
    # Optional schema for the format of observed data
    observation_schema = {'t': {'min': 0, 'required': True},
                          'x': {'units': True, 'required': True},
                          'y': {'units': True, 'required': True},
                          'phi': {'required': False}}
    
    def validate_observation(self, observation):
        """Additional checks on the observation"""
        assert isinstance(observation['t'], datetime)
        return observation
        
    # Optional schema for the format of test parameters
    params_schema = {'rotate': {'required': False}}

    # Optional schema for the format of default test parameters
    default_params = {'rotate': False}
    
    def compute_score(self, observation, prediction):
        """Optionally use additional information to compute model/data agreement"""
        observation_rotated = observation.copy()
        if 'phi' in observation:
            observation_rotated['x'] *= cos(observation['phi'])
            observation_rotated['y'] *= cos(observation['phi'])
        return super().compute_score(observation_rotated, prediction)
            

### Now we can instantiate a test.  Each test instance is a combination of the test class, describing the testing logic and required capabilties, plus some <i>'observation'</i>, i.e. data.  

In [53]:
# N.B.: This data is made up
earth_position_test_march = StricterPositionTest(name = "Earth Orbital Data on March 1st, 2019",
                                       observation = {'t': datetime(2019, 3, 1),
                                                      'x': 7.905e7 * pq.km, 
                                                      'y': 1.254e8 * pq.km})

### Finally, we can execute this one test against this one model

In [54]:
score = earth_position_test_march.judge(earth_model)
score

Pass

### And we can get additional information about the test, including intermediate objects computed in order to generate a score.

In [55]:
score.describe()

In [56]:
score.prediction, score.observation

({'t': datetime.datetime(2019, 3, 1, 0, 0),
  'x': array(79046604.57417324) * km,
  'y': array(1.25401809e+08) * km},
 {'t': datetime.datetime(2019, 3, 1, 0, 0),
  'x': array(79050000.) * km,
  'y': array(1.254e+08) * km})

In [57]:
score.get_raw()

array(3847.28076371) * km

### We may want to bundle many such tests into a `TestSuite`.  This suite may contain test from multiple classes, or simply tests which differ only in the observation (data) used to instantiate them.

In [58]:
earth_position_test_april = StricterPositionTest(name = "Earth Orbital Data on April 1st, 2019",
                                       observation = {'t': datetime(2019, 4, 1),
                                                      'x': 160000 * pq.km, 
                                                      'y': 70000 * pq.km})

earth_position_suite = sciunit.TestSuite([earth_position_test_march,
                                         earth_position_test_april],
                                         name = 'Earth observations in 2019')

### We can then test our model against this whole suite of tests

In [59]:
scores = earth_position_suite.judge(earth_model)

### Rich HTML output is automatically produced when this score output is summarized

In [60]:
scores

,"Earth Orbital Data on March 1st, 2019","Earth Orbital Data on April 1st, 2019"
Kepler's Earth Model,Pass,Fail


### We can then expand this to multiple models

In [61]:
# Just like the Kepler model, but returning a random orbital angle
class RandomModel(KeplerModel):
    def get_position(self, t):
        r, theta = super().get_position(t)
        return r, 2*pi*np.random.rand()

In [62]:
random_model = RandomModel(name = "Random Earth Model",
                                 semimajor_axis=149598023 * pq.km, 
                                 semiminor_axis=149577161 * pq.km, 
                                 period=timedelta(365, 22118), # Period of Earth's orbit 
                                 perihelion=datetime(2019, 1, 3, 0, 19), # Time and date of Earth's last perihelion
                                 )

In [63]:
scores = earth_position_suite.judge([earth_model, random_model])

In [64]:
scores

,"Earth Orbital Data on March 1st, 2019","Earth Orbital Data on April 1st, 2019"
Kepler's Earth Model,Pass,Fail
Random Earth Model,Fail,Fail


### <i>SciUnit</i> is in use in several multiscale modeling projects including:
#### - [The Human Brain Project](https://www.humanbrainproject.eu/en/) (neurophysiology, neuroanatomy, neuroimaging)
#### - [OpenWorm](http://openworm.org/) (physiology, behavior)

### <i>[NeuronUnit](http://neuronunit.scidash.org)</i> is a reference implementation in the domain of neurophysiology of:
#### - model classes
#### - test classes
#### - capability classes
#### - tools for constructing tests from several public neurophysiology databases
#### - tools for implementing capabilities from standard model exchange formats
#### - tools for executing simulations underlying testing using popular simulators
#### - test-driven model optimization

### <i>[SciDash](http://dash.scidash.org)</i> is a web application for creating, scheduling, and viewing the results of SciUnit tests without writing a single line of code.

<hr>
<div align='center' style='font-size:300%; line-height:30px;'>
Links:
</div>
<table align='center' style='font-size:150%; line-height:30px;'>
<tr>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/SciUnit/sci-unit-wide.png?raw=true"  href="http://sciunit.scidash.org" width="50%"></td>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/neuronunit-logo.png?raw=trueg" href="http://neuronunit.scidash.org" width="50%"></td>
</tr>
<tr>
    <td width='25%'><img src="https://github.com/scidash/assets/blob/master/logos/scidash_logo.png?raw=true"  href="http://dash.scidash.org" width="50%"></td>
    <td width='25%'><img src="http://science-marketplace.org/oc-content/uploads/6/599.png" width="35%"></td>
</tr>
</table>

<hr>
<div align='center' style='font-size:200%; line-height:30px;'>
Funded by:
</div>
<table align='center' style='line-height:30px;'>
<tr>
    <td width='25%'>R01DC018455 (NIDCD)</td>
    <td width='25%'><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/US-NIH-NIDCD-Logo.svg/1920px-US-NIH-NIDCD-Logo.svg.png" width="50%"></td>
    <td width='25%'>R01MH106674 (NIMH)</td>
    <td width='25%'><img src="https://upload.wikimedia.org/wikipedia/commons/a/a0/NIH-NIMH-logo-new.png" width="100%"></td>
</tr>
<tr>
    <td>R01EB021711 (NIBIB)</td>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/1/15/NIH_NIBIB_Vertical_Logo_2Color.jpg" width="50%"></td>
    <td>Human Brain Project</td>
    <td><img src="https://pbs.twimg.com/profile_images/660035391442042880/v7RkSosC_400x400.png" width="50%"></td>
</tr>
</table>

### Thanks also to Sharon Crook, Justas Birgiolas, Russell Jarvis, and Cyrus Omar